## Realisation de ce [tutoriel_1](https://victorzhou.com/blog/keras-cnn-tutorial/#2-preparing-the-data) et ce [tutoriel_2](https://elitedatascience.com/keras-tutorial-deep-learning-in-python)

In [1]:
import numpy as np

import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

from keras.utils import np_utils

print('env: ', os.environ['CONDA_DEFAULT_ENV'])

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
env:  deep-learning


## Lecture des csv

In [2]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

print(
    train_images.dtype, '\n',
    test_images.dtype, '\n',
)

uint8 
 uint8 



## Preprocessing

Soit 85% de données d'entrainement

Soit 15% de données de test

In [3]:
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# train_images = train_images.astype('float32')
# test_images = test_images.astype('float32')

print(
    train_images.shape, '\n',
    test_images.shape, '\n'
)

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(
    train_images.shape, '\n',
    train_labels.shape, '\n',
    test_images.shape, '\n',
    test_labels.shape, '\n'
)

print(
    train_images.dtype, '\n',
    test_images.dtype, '\n',
)

(60000, 28, 28) 
 (10000, 28, 28) 

(60000, 28, 28, 1) 
 (60000,) 
 (10000, 28, 28, 1) 
 (10000,) 

float64 
 float64 



## Construction du modèle

In [4]:
num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

optimizer = 'adam'

model.compile(
  optimizer,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

## Resultat entrainement 
```
num_filters:	 8 
filter_size:	 3 
pool_size:	 2 
optimizer:	 adam 
nb_epoch:	 3
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 10s 171us/sample - loss: 0.3605 - accuracy: 0.8956 - val_loss: 0.2344 - val_accuracy: 0.9335
Epoch 2/3
60000/60000 [==============================] - 9s 156us/sample - loss: 0.1900 - accuracy: 0.9444 - val_loss: 0.1546 - val_accuracy: 0.9538
Epoch 3/3
60000/60000 [==============================] - 9s 157us/sample - loss: 0.1403 - accuracy: 0.9592 - val_loss: 0.1233 - val_accuracy: 0.9610
```

In [5]:
nb_epoch = 3

print(
  'num_filters:\t', num_filters,
  '\nfilter_size:\t', filter_size,
  '\npool_size:\t', pool_size,
  '\noptimizer:\t', optimizer,
  '\nnb_epoch:\t', nb_epoch,
)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.fit(
  train_images,
  train_labels,
  epochs=nb_epoch,
  validation_data=(test_images, test_labels ),
)

# Save the model to disk.
model.save_weights('tuto1_cnn.h5')

num_filters:	 8 
filter_size:	 3 
pool_size:	 2 
optimizer:	 adam 
nb_epoch:	 3
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 12s 202us/sample - loss: 0.3665 - accuracy: 0.8951 - val_loss: 0.2144 - val_accuracy: 0.9396
Epoch 2/3
60000/60000 [==============================] - 11s 178us/sample - loss: 0.1786 - accuracy: 0.9491 - val_loss: 0.1380 - val_accuracy: 0.9590
Epoch 3/3
60000/60000 [==============================] - 11s 181us/sample - loss: 0.1284 - accuracy: 0.9632 - val_loss: 0.1250 - val_accuracy: 0.9630


val_loss: 0.1233 - val_accuracy: 0.9610

```
print(model.evaluate(test_images, to_categorical(test_labels), verbose=0))
> [0.12332842820584775, 0.961]
```

In [6]:
# to_categorical(test_labels)

print(model.evaluate(test_images, to_categorical(test_labels), verbose=0))

[0.23840623259051935, 0.9496]


In [14]:
# Load the model from disk later using:
# model.load_weights('tutoriels_CNN/tuto1_cnn.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:37])

print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

print(test_labels[:37]) # [7, 2, 1, 0, 4]

print(predictions == test_labels[:37])

[7 2 1 0 4 1 4 4 6 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7]
False


# Les hyperparamètres à tester:

## Network Depth
What happens if we add or remove Convolutional layers? How does that affect training and/or the model’s final performance?
```
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),

  Conv2D(num_filters, filter_size),
  
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])
```

## Dropout
What if we tried adding Dropout layers, which are commonly used to prevent overfitting (surentrainement) ?
```
from tensorflow.keras.layers import Dropout

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  
  Dropout(0.5),

  Flatten(),
  Dense(10, activation='softmax'),
])
```

## Fully-connected Layers
What if we add fully-connected layers between the Convolutional outputs and the final Softmax layer? This is something commonly done in CNNs used for Computer Vision.

```
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),

  Dense(64, activation='relu'),
  
  Dense(10, activation='softmax'),
])
```

## Convolution Parameters

What if we play with the Conv2D parameters? For example:

```
Conv2D(
    num_filters,
    filter_size,
    input_shape=(28, 28, 1),

    strides=2,
    padding='same',
    activation='relu',
    
  ),
```

code complet du tutoriel :

``` python
# The full CNN code!
####################
import numpy as np
import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

num_filters = 8
filter_size = 3
pool_size = 2

# Build the model.
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=3,
  validation_data=(test_images, to_categorical(test_labels)),
)

# Save the model to disk.
model.save_weights('cnn.h5')

# Load the model from disk later using:
# model.load_weights('cnn.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]
```